In [6]:
import os
import random
import numpy as np
import tqdm
from rouge_score import rouge_scorer
import utils
import torch
import transformers
from torch.utils.data import Dataset
import openai
from dotenv import load_dotenv

In [3]:
load_dotenv(override=True)
openai.api_key = os.getenv("OPENAI_API_KEY")

In [7]:
seed_task = {
    "instruction": "Write a Python program to compute the sum of integers from k to n.", 
    "output": "def sum(k, n):\n    sum = 0\n    for i in range(k, n+1):\n        sum += i\n    return sum"
}

In [5]:
methods = [
    'Add new constraints and requirements to the original problem, adding approximately 10 additional words.',
    'Replace a commonly used requirement in the programming task with a less common and more specific one.',
    'If the original problem can be solved with only a few logical steps, please add more reasoning steps.',
    'Provide a piece of erroneous code as a reference to increase misdirection.',
    'Propose higher time or space complexity requirements, but please refrain from doing so frequently.'
]

#### Step 1. Evolve Instruction

Let's evolve the initial instruction with a randomly chosen evolution method

In [8]:
chosen_method = random.choice(methods)
prompt_template = f"Please increase the difficulty of the given programming test question a bit.\n\nYou can increase the difficulty using, but not limited to, the following methods:\n{chosen_method}\n\n{seed_task['instruction']}"

completion = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "user", "content": prompt_template}
  ]
)

In [11]:
print(f'PROMPT: {prompt_template}')
print(f'RESPONSE: {completion.choices[0].message.content}')

PROMPT: Please increase the difficulty of the given programming test question a bit.

You can increase the difficulty using, but not limited to, the following methods:
Propose higher time or space complexity requirements, but please refrain from doing so frequently.

Write a Python program to compute the sum of integers from k to n.
RESPONSE: Write a Python program to compute the sum of all odd integers from k to n.


#### Step 2. Generate a response

Now let's generate a response for the evolved instruction

In [12]:
response = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "user", "content": completion.choices[0].message.content}
  ]
)

In [13]:
print(response.choices[0].message.content)

Here is a Python program that computes the sum of all odd integers from k to n:

```python
def sum_of_odd_integers(k, n):
    total = 0
    for num in range(k, n + 1):
        if num % 2 != 0:  # check if the number is odd
            total += num
    return total

# Example usage
k = 1
n = 10
print("Sum of odd integers from", k, "to", n, "is:", sum_of_odd_integers(k, n))
```

Output:
```
Sum of odd integers from 1 to 10 is: 25
```

In the above program, the `sum_of_odd_integers` function takes two parameters `k` and `n`, which represent the range of integers to be considered. It initializes `total` to 0, and then iterates through each number from `k` to `n`. If the number is odd (i.e., not divisible by 2), it adds it to the `total`. Finally, it returns the `total` as the sum of all odd integers in the range.


In [15]:
evolved_instruction1 = {
    "instruction": completion.choices[0].message.content,
    "output": response.choices[0].message.content
}

#### Step 3. Evaluate that the evolved instruction has information gain

Now let's evaluate the instruction to make sure it is providing information gain

In [23]:
evaluation_prompt = f'Here are two Instructions to ChatGPT AI, do you think they are equal to each other, which meet the following requirements:\n1. They have the same constraints and requirements.\n2. They have the same depth and breadth\nThe First Prompt: {seed_task["instruction"]}\nThe Second Prompt: {evolved_instruction1["instruction"]}\nYour Judgement (Just answer: Equal or Not Equal. No need to explain the reason):'

evaluation = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "user", "content": evaluation_prompt}
  ]
)


In [24]:
evaluation = evaluation.choices[0].message.content
print(evaluation)

Not equal.


#### Step 4. Evolving the previous instruction

Now we're going to repeat the previous step this time starting with the evolution

In [25]:
chosen_method2 = random.choice(methods)
prompt_template2 = f"Please increase the difficulty of the given programming test question a bit.\n\nYou can increase the difficulty using, but not limited to, the following methods:\n{chosen_method2}\n\n{evolved_instruction1['instruction']}"

completion2 = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "user", "content": prompt_template2}
  ]
)
completion2 = completion2.choices[0].message.content

In [27]:
print(prompt_template2)
print(completion2)

Please increase the difficulty of the given programming test question a bit.

You can increase the difficulty using, but not limited to, the following methods:
Provide a piece of erroneous code as a reference to increase misdirection.

Write a Python program to compute the sum of all odd integers from k to n.
Write a Python program to compute the sum of all odd integers from k to n. However, do not use any loops or conditional statements (if, elif, else) in your solution. You can only use mathematical operations and built-in functions.


In [28]:
# Now generate the response
response2 = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "user", "content": completion2}
  ]
)
response2 = response2.choices[0].message.content

In [29]:
print(completion2)
print(response2)

Write a Python program to compute the sum of all odd integers from k to n. However, do not use any loops or conditional statements (if, elif, else) in your solution. You can only use mathematical operations and built-in functions.
To compute the sum of all odd integers from `k` to `n` without using any loops or conditional statements, we can make use of the mathematical properties of arithmetic series.

The mathematical formula to compute the sum of an arithmetic series is:

```
sum = (n / 2) * (first_term + last_term)
```

Since we want to sum only odd integers, we can skip the even numbers by adjusting the values of `k` and `n` as follows:
- If `k` is even, we increment it by 1 to make it odd.
- If `n` is even, we decrement it by 1 to make it odd.

Here's the Python code to solve the problem:

```python
def sum_of_odd_integers(k, n):
    if k % 2 == 0:
        k += 1
    if n % 2 == 0:
        n -= 1

    num_terms = (n - k) // 2 + 1
    sum = num_terms * (k + n) // 2
    return sum


#### Step 5. Evolve One more time

In [30]:
evolved_instruction2 = {
    "instruction": completion2,
    "output": response2
}

In [31]:
chosen_method3 = random.choice(methods)
prompt_template3 = f"Please increase the difficulty of the given programming test question a bit.\n\nYou can increase the difficulty using, but not limited to, the following methods:\n{chosen_method3}\n\n{evolved_instruction2['instruction']}"

completion3 = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "user", "content": prompt_template3}
  ]
)
completion3 = completion3.choices[0].message.content

In [32]:
print(prompt_template3)
print(completion3)

Please increase the difficulty of the given programming test question a bit.

You can increase the difficulty using, but not limited to, the following methods:
Provide a piece of erroneous code as a reference to increase misdirection.

Write a Python program to compute the sum of all odd integers from k to n. However, do not use any loops or conditional statements (if, elif, else) in your solution. You can only use mathematical operations and built-in functions.
Write a Python program to compute the sum of all odd integers from k to n. However, do not use any loops, conditional statements (if, elif, else), or built-in mathematical functions (e.g., sum, range) in your solution. You can only use mathematical operations and built-in functions. Also, ensure that your program has a time complexity of O(1).


In [33]:
response3 = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "user", "content": completion3}
  ]
)
response3 = response3.choices[0].message.content

In [34]:
print(response3)

To compute the sum of all odd integers from k to n without using loops, conditional statements, or built-in mathematical functions, you can use the formula for the sum of an arithmetic series.

The formula for the sum of an arithmetic series is: 
sum = (n/2) * (a + b), 
where n is the number of terms, a is the first term, and b is the last term.

In this case, we want to compute the sum of the odd integers from k to n. To do this, we need to adjust the formula to account for the fact that n may not be an odd number.

Here's the Python code to compute the sum of all odd integers from k to n:

```python
def sum_of_odd_integers(k, n):
    if k % 2 == 0:
        a = k + 1
    else:
        a = k
        
    if n % 2 == 0:
        b = n - 1
    else:
        b = n
        
    num_of_terms = (b - a) // 2 + 1
    return num_of_terms * (a + b) // 2
```

The time complexity of this solution is O(1) because it does not involve any loops or iterative processes.


In [35]:
evolved_instruction3 = {
    "instruction": completion3,
    "output": response3
}

In [40]:
evaluation_prompt3 = f'Here are two Instructions to ChatGPT AI, do you think they are equal to each other, which meet the following requirements:\n1. They have the same constraints and requirements.\n2. They have the same depth and breadth\nThe First Prompt: {evolved_instruction2["instruction"]}\nThe Second Prompt: {evolved_instruction3["instruction"]}\nYour Judgement (Just answer: Equal or Not Equal. No need to explain the reason):'

evaluation3 = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "user", "content": evaluation_prompt3}
  ]
)
evaluation3 = evaluation3.choices[0].message.content
print(evaluation3)

Not equal.
